<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Проверка-модели" data-toc-modified-id="Проверка-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка модели</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></div>

# Проект для «Викишоп» - поиск токсичных комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75.

**Цель проекта**

Обучить модель для классификации комментариев на позитивные и негативные.

**Описание данных**

Датасет с разметкой о токсичности правок. Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Установим пакет CatBoost:

In [3]:
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.7 MB/s eta 0:00:00


Импортируем необходимые библиотеки:

In [4]:
import pandas as pd

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Считаем данные из csv-файла в датафрейм, сохраним в переменную и выведем ее на экран:

In [5]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Выведем основную информацию о датафрейме на экран:

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Проверим датасет на количество дубликатов:

In [7]:
data.duplicated().sum()

0

Создадим новый столбец lemm_text:

In [8]:
data['lemm_text'] = data['text']

Выведем обновленный датасет на экран:

In [9]:
data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation\nWhy the edits made under my usern...
1,1,D'aww! He matches this background colour I'm s...,0,D'aww! He matches this background colour I'm s...
2,2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man, I'm really not trying to edit war. It..."
3,3,"""\nMore\nI can't make any real suggestions on ...",0,"""\nMore\nI can't make any real suggestions on ..."
4,4,"You, sir, are my hero. Any chance you remember...",0,"You, sir, are my hero. Any chance you remember..."


Создадим функцию для лемматизации текста, применим ее к столбцу lemm_text и вновь взглянем на обновленный датасет:

In [10]:
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    words = text.split()
    words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(words)

data['lemm_text'] = data['lemm_text'].apply(lemmatize_words)

data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,D'aww! He match this background colour I'm see...
2,2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man, I'm really not try to edit war. It's ..."
3,3,"""\nMore\nI can't make any real suggestions on ...",0,""" More I can't make any real suggestions on im..."
4,4,"You, sir, are my hero. Any chance you remember...",0,"You, sir, be my hero. Any chance you remember ..."


Напишем и применим функцию для очистки текста от лишних символов:

In [11]:
def clear_text(text):

    clear_text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text = clear_text.split()
    clear_text = " ".join(clear_text)
    return clear_text

data['lemm_text'] = data['lemm_text'].apply(clear_text)

data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not try to edit war It s ju...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


Приведем столбец lemm_text к нижнему регистру и выведем обновленный датасет на экран:

In [12]:
data['lemm_text'] = data['lemm_text'].str.lower()

data.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Разобьем датасет на новый набор данных data1 и тестовую выборку:

In [13]:
data1, test = train_test_split(data, test_size=0.2)

Разобьем data1 на обучающую и валидационную выборки:

In [14]:
train, valid = train_test_split(data1, test_size=0.25)

Выделим из всех выборок признаки и целевой признак:

In [15]:
features_train = train.drop(['toxic', 'Unnamed: 0'], axis=1)
target_train = train['toxic']

In [16]:
features_valid = valid.drop(['toxic', 'Unnamed: 0'], axis=1)
target_valid = valid['toxic']

In [17]:
features_test = test.drop(['toxic', 'Unnamed: 0'], axis=1)
target_test = test['toxic']

**Вывод:** В ходе предобработки датасет проверен на наличие дубликатов, создан новый столбец lemm_text, проведена его лемматизация и очистка от лишних символов; также, он приведен к нижнему регистру. Датасет поделен на обучающую, валидационную и тестовую выборки. Выделены признаки и целевой признак.

## Обучение моделей

Произведем оценку важности слов столбца lemm_text на всех выборках, с учетом стоп-слов:

In [18]:
count_tf_idf_train = TfidfVectorizer(stop_words=list(stopwords))
tf_idf_train = count_tf_idf_train.fit_transform(train.lemm_text)

In [19]:
tf_idf_valid = count_tf_idf_train.transform(valid.lemm_text)

In [20]:
tf_idf_test = count_tf_idf_train.transform(test.lemm_text)

Создадим цикл и определим регуляризацию для модели логистической регрессии при наилучшем значении F1-меры:

In [21]:
best_model = None
best_result = 0
best_depth = 0

for c in range(1, 20):
    model = LogisticRegression(random_state=12345, max_iter=500, C=c)   # зададим число итераций, чтобы избежать ошибки при выводе результата
    model.fit(tf_idf_train, target_train)
    predictions_valid = model.predict(tf_idf_valid)
    result = f1_score(target_valid, predictions_valid)

    if result > best_result:
        best_model = model
        best_result = result
        best_C = c

print("F1 лучшей модели:", best_result, "Регуляризация:", best_C)

F1 лучшей модели: 0.7755173623290073 Регуляризация: 11


Создадим цикл и определим глубину дерева для модели дерева решений при наилучшем значении F1-меры:

In [22]:
best_model = None
best_result = 0
best_depth = 0

for depth in range(1, 90):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(tf_idf_train, target_train)
    predictions_valid = model.predict(tf_idf_valid)
    result = f1_score(target_valid, predictions_valid)

    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

print("F1 лучшей модели:", best_result, "Глубина дерева:", best_depth)

F1 лучшей модели: 0.723024054982818 Глубина дерева: 89


Создадим цикл и определим количество оценщиков для модели случайного леса при наилучшем значении F1-меры:

In [23]:
best_model = None
best_result = 0
best_estimators = 0

for est in range(1, 30):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(tf_idf_train, target_train)
    predictions_valid = model.predict(tf_idf_valid)
    result = f1_score(target_valid, predictions_valid)

    if result > best_result:
        best_model = model
        best_result = result
        best_estimators = est

print("F1 лучшей модели:", best_result, "Количество оценщиков:", best_estimators)

F1 лучшей модели: 0.7067437379576108 Количество оценщиков: 29


Обучим модель на основе алгоритма LGBMClassifier и определим для нее значение F1-меры:

In [24]:
model = LGBMClassifier()

model.fit(tf_idf_train, target_train)
predictions_valid = model.predict(tf_idf_valid)
print("F1 модели:", f1_score(target_valid, predictions_valid))

[LightGBM] [Info] Number of positive: 9848, number of negative: 85726
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 14.877735 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 467243
[LightGBM] [Info] Number of data points in the train set: 95574, number of used features: 9346
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.103041 -> initscore=-2.163888
[LightGBM] [Info] Start training from score -2.163888
F1 модели: 0.7451695224207072


Обучим модель на основе алгоритма CatBoostClassifier и определим для нее значение F1-меры:

In [25]:
model = CatBoostClassifier()

model.fit(tf_idf_train, target_train)
predictions_valid = model.predict(tf_idf_valid)
print("F1 модели:", f1_score(target_valid, predictions_valid))

Learning rate set to 0.0722
0:	learn: 0.6189007	total: 2.46s	remaining: 40m 56s
1:	learn: 0.5578952	total: 4.88s	remaining: 40m 35s
2:	learn: 0.5078496	total: 6.5s	remaining: 36m
3:	learn: 0.4659218	total: 8.12s	remaining: 33m 42s
4:	learn: 0.4298834	total: 9.74s	remaining: 32m 18s
5:	learn: 0.3989063	total: 11.4s	remaining: 31m 30s
6:	learn: 0.3733248	total: 13s	remaining: 30m 47s
7:	learn: 0.3525080	total: 14.8s	remaining: 30m 36s
8:	learn: 0.3344177	total: 17.4s	remaining: 31m 59s
9:	learn: 0.3191026	total: 19.1s	remaining: 31m 26s
10:	learn: 0.3061288	total: 20.7s	remaining: 31m
11:	learn: 0.2955477	total: 22.3s	remaining: 30m 37s
12:	learn: 0.2850560	total: 24s	remaining: 30m 20s
13:	learn: 0.2771683	total: 25.6s	remaining: 30m 4s
14:	learn: 0.2705985	total: 27.3s	remaining: 29m 54s
15:	learn: 0.2647067	total: 30s	remaining: 30m 47s
16:	learn: 0.2593571	total: 31.7s	remaining: 30m 31s
17:	learn: 0.2546817	total: 33.3s	remaining: 30m 16s
18:	learn: 0.2508565	total: 34.9s	remaining:

**Вывод:** Для выполнения задачи проекта были обучены следующие модели: LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, LGBMClassifier и CatBoostClassifier. Для всех них было определено значение метрики F1. Наилучшим образом себя показала логистическая регрессия с регуляризацией, равной 11-ти. F1-мера для нее составила 0,78.

## Проверка модели

Проверим модель с наилучшей метрикой на тестовой выборке:

In [27]:
model = LogisticRegression(random_state=12345, max_iter=500, C=11)

model.fit(tf_idf_train, target_train)
predictions = model.predict(tf_idf_test)
print('F1 модели:', f1_score(target_test, predictions))

F1 модели: 0.774159478964971


**Вывод:** Значение F1-меры больше 0,75, соответственно, данная модель подходит для выполнения задачи проекта.

## Общий вывод

 В ходе предобработки датасет проверен на наличие дубликатов, создан новый столбец lemm_text, проведена его лемматизация и очистка от лишних символов; также, он приведен к нижнему регистру. Датасет поделен на обучающую, валидационную и тестовую выборки. Выделены признаки и целевой признак.

 Для выполнения задачи проекта были обучены следующие модели: LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, LGBMClassifier и CatBoostClassifier. Для всех них было определено значение метрики F1. Наилучшим образом себя показала логистическая регрессия с регуляризацией, равной 11-ти. F1-мера для нее составила 0,78.

 При проверке на тестовой выборке, значение F1-меры составило 0,77. Следовательно, данная модель подходит для выполнения задачи проекта.